Імпортуємо бібліотеки

In [1]:
import os
import tensorflow as tf
import numpy as np
import cv2
import pandas as pd

Змінюємо розмір зображень

In [2]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255,
    validation_split=0.3
)

Завантажуємо дані

In [3]:
train_generator = datagen.flow_from_directory(
    'cars',
    target_size=(150,150),
    batch_size=32,
    shuffle=True,
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'cars',
    target_size=(150,150),
    batch_size=32,
    shuffle=True,
    subset='validation'
)

Found 8 images belonging to 2 classes.
Found 2 images belonging to 2 classes.


Створюємо модель Xception

In [4]:
pre_trained_model = tf.keras.applications.xception.Xception(
    input_shape=(150, 150, 3),
    include_top=False,
    weights='imagenet'
)

for layer in pre_trained_model.layers:
    layer.trainable = False

Подивимось на архітектуру моделі

In [5]:
pre_trained_model.summary()

Model: "xception"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 74, 74,    │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 74, 74,    │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 74, 74,    │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 72, 72,    │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 72, 72,    │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 72, 72,    │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 72, 72,    │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 72, 72,    │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 72, 72,    │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 72, 72,    │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 72, 72,    │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 36, 36,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 36, 36,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 36, 36,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 36, 36,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 36, 36,    │          0 │ add[0][0]       

 Total params: 20,861,480 (79.58 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,861,480 (79.58 MB)

Додаємо до моделі останній шар

In [6]:
last_layer = pre_trained_model.get_layer('block14_sepconv2_act')
last_layer_output = last_layer.output

Створюємо нову модель

In [7]:
x = tf.keras.layers.Flatten()(last_layer_output)
x = tf.keras.layers.Dense(2024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)

output = tf.keras.layers.Dense(2, activation='softmax')(x)

model = tf.keras.Model(pre_trained_model.input, output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 150, 150,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 74, 74,    │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 74, 74,    │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 74, 74,    │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 72, 72,    │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 72, 72,    │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 72, 72,    │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 72, 72,    │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 72, 72,    │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 72, 72,    │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 72, 72,    │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 72, 72,    │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 36, 36,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 36, 36,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 36, 36,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 36, 36,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 36, 36,    │          0 │ add[0][0]       

 Total params: 124,496,354 (474.92 MB)

 Trainable params: 103,634,874 (395.34 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

Компілюємо модель

In [8]:
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.000001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Навчаємо модель

In [9]:
import sys
from PIL import Image
sys.modules['Image'] = Image 

In [10]:
from PIL import Image
print(Image.__file__)


/Users/mrpaschenko/Developer/dinns-labs/lab6/.venv/lib/python3.12/site-packages/PIL/Image.py


In [11]:
model.fit(train_generator, epochs=15, validation_data=validation_generator)

/Users/mrpaschenko/Developer/dinns-labs/lab6/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
/Users/mrpaschenko/Developer/dinns-labs/lab6/.venv/lib/python3.12/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.8750 - loss: 0.4779 - val_accuracy: 0.5000 - val_loss: 0.5368
Epoch 2/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.2676 - val_accuracy: 0.5000 - val_loss: 0.5137
Epoch 3/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.2434 - val_accuracy: 0.5000 - val_loss: 0.4990
Epoch 4/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.2411 - val_accuracy: 0.5000 - val_loss: 0.4818
Epoch 5/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1871 - val_accuracy: 0.5000 - val_loss: 0.4702
Epoch 6/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 1.0000 - loss: 0.1498 - val_accuracy: 0.5000 - val_loss: 0.4603
Epoch 7/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1449 - val_accuracy: 0.5000 - val_loss: 0.4600
Epoch 8/15
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 1.0000 - loss: 0.1619 - val_accuracy: 0.5000 - val_loss: 0.4543
Epoch 9/15
1/1 ━

Виконуємо прогноз на валідаційній вибірці

In [12]:
model.predict(validation_generator)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step


array([[0.5052341 , 0.49476588],
       [0.85284907, 0.1471509 ]], dtype=float32)

Перевіряємо модель на зображеннях

In [13]:
files_names = os.listdir('images')
names = ['ford', 'notford']
classes = []
i = 1

for item in files_names:
  img_path = os.path.join('images', item)

  img = tf.keras.utils.load_img(img_path, target_size=(150, 150))
  img = tf.keras.utils.img_to_array(img) / 255
  img = np.expand_dims(img, axis=0)
  img = np.vstack([img])

  pred = model.predict(img, batch_size=10)

  num = np.argmax(pred)

  print(f'image {i} ({files_names[i-1]}) is {names[num]}')

  i += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step
image 1 (ford 4.jpg) is ford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
image 2 (ford 2.png) is ford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
image 3 (ford 3.jpg) is ford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
image 4 (ford 1.png) is ford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
image 5 (notford 1.png) is notford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
image 6 (notford 3.png) is notford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
image 7 (notford 2.png) is notford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
image 8 (notford 6.png) is notford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
image 9 (notford 5.png) is notford
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
image 10 (notford 4.png) is notford


Створюємо функцію для передбачення зображення

In [14]:
def predict_from_image(img_path):
  TestData = []

  img = tf.keras.utils.load_img(img_path,target_size=(150,150)).convert('RGB')
  imgarray = np.asarray(img)
  TestData.append(imgarray)
  Test_X=np.array(TestData)
  Test_X = Test_X.astype('float32')
  Test_X /=255

  return model.predict(Test_X)[0][0]

Перевіряємо модель на відео

In [15]:
video_path = "ford.mp4"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
  print("error")

frame_no = 0
hasLogo = False
appearCount = 0
dates = []
No_logo_in_video = True
DFVideo = pd.DataFrame(columns = ["FrameNo", "Probability"])
count = 0

Detected = False
first_run = True
print("start")

while cap.isOpened():
  hasLogo = False
  frame_exists, curr_frame = cap.read()

  if frame_exists:
    if frame_no % 2 == 0:
      cv2.imwrite("frame%d.jpg" % frame_no, curr_frame)
      probability = predict_from_image("frame%d.jpg" % frame_no)
      print(probability)
      timeF = cap.get(cv2.CAP_PROP_POS_MSEC)
      timeF = timeF / 1000

      if probability < 0.5:
       hasLogo = False

       if not hasLogo:
         #print("not detec")

         if not first_run:
           if not Detected:
             Detected = True

      if probability >= 0.5:
        hasLogo = True
        first_run = False
        No_logo_in_video = False
        #print("detec")
        if hasLogo:
          appearCount +=1
          dates.append(str(round(timeF, 2)))

          if Detected:
            Detected = False

        os.remove("frame%d.jpg" % frame_no)
        hasLogo = False

      DFVideo.loc[count] = ["frame%d.jpg" % frame_no, probability]
      count = count+1
    
  else:
    break
  frame_no +=1

if No_logo_in_video:
  print("No logo")

print("Всього кадрів: "+str(appearCount))
print(dates)

start
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
0.93628395
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
0.93612087
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
0.93612224
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
0.93619484
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
0.93618584
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
0.93620557
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
0.9362448
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
0.9362448
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
0.9361904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
0.9361904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
0.93624926
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
0.93624926
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
0.93624926
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
0.9362516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
0.9362516
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
0.93625677
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
0.93626827
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
0.9362705
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
0.9362244
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
0.9362244
1/1 ━━━━━━━━━━━━━━━━━━━

In [17]:
model.save('model.keras')